# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_path = "Output/city_data.csv" 
city_data_df = pd.read_csv(city_data_path)
city_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Palana,100,RU,1584137477,77,59.12,159.97,12.29,9.13
1,Jamestown,75,US,1584137478,55,42.10,-79.24,44.60,24.16
2,Kaeo,1,NZ,1584137479,77,-35.10,173.78,66.99,4.00
3,Ballina,8,AU,1584137480,82,-28.87,153.57,68.00,5.82
4,Ushuaia,20,AR,1584137481,71,-54.80,-68.30,53.60,4.70
...,...,...,...,...,...,...,...,...,...
544,Nyurba,0,RU,1584138128,94,63.28,118.33,-21.91,4.50
545,Luchenza,97,MW,1584138129,94,-16.01,35.31,65.75,3.53
546,Kallaste,66,EE,1584138130,72,58.66,27.16,32.00,6.93
547,Kolosovka,100,RU,1584138131,86,56.47,73.61,26.69,13.98


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#build map figure
gmaps.configure(api_key=g_key)
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "TERRAIN")

In [4]:
#get lats and longs as lists
lat_list = city_data_df["Lat"].to_list()
long_list = city_data_df["Lng"].to_list()
#zip list into tuple pairs
coords_zip = zip(lat_list, long_list)
#turn zip object into normal list
coords_list = [coord for coord in coords_zip]

In [5]:
humidity = city_data_df["Humidity"]
heat_layer = gmaps.heatmap_layer(coords_list, weights = humidity)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#assign variables
temp = city_data_df["Max Temp"]
w_spd = city_data_df["Wind Speed"]
humdty = city_data_df["Humidity"]
clouds = city_data_df["Cloudiness"]

#grab values    
vacay_df = city_data_df.loc[(clouds <= 30) &
((temp <= 90) & (temp >= 70)) &
((humdty >= 35) & (humdty <= 60)) &
(w_spd <=15)]
vacay_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
20,Castro,0,BR,1584137354,47,-24.79,-50.01,75.83,3.60
45,Mar del Plata,0,AR,1584137540,42,-38.00,-57.56,84.20,9.17
122,Praia,8,CV,1584137636,44,14.92,-23.51,78.80,14.99
277,Point Fortin,20,TT,1584137823,51,10.18,-61.68,86.00,14.99
309,Palāsa,0,IN,1584137857,52,18.77,84.42,73.49,1.83
364,Mount Isa,0,AU,1584137920,46,-20.73,139.50,73.40,6.93
366,Valparaíso,10,CL,1584137717,49,-33.04,-71.63,73.40,13.87
391,Gladstone,0,AU,1584137949,57,-23.85,151.25,75.20,10.29
394,Karasburg,0,NaN,1584137953,44,-28.02,18.75,70.79,11.99
440,Hualmay,30,PE,1584138006,54,-11.10,-77.61,80.69,10.20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotels_df = pd.DataFrame(vacay_df)
hotels_df = hotels_df.reset_index(drop=True)

In [17]:
hotels_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Castro,0,BR,1584137354,47,-24.79,-50.01,75.83,3.60
1,Mar del Plata,0,AR,1584137540,42,-38.00,-57.56,84.20,9.17
2,Praia,8,CV,1584137636,44,14.92,-23.51,78.80,14.99
3,Point Fortin,20,TT,1584137823,51,10.18,-61.68,86.00,14.99
4,Palāsa,0,IN,1584137857,52,18.77,84.42,73.49,1.83
5,Mount Isa,0,AU,1584137920,46,-20.73,139.50,73.40,6.93
6,Valparaíso,10,CL,1584137717,49,-33.04,-71.63,73.40,13.87
7,Gladstone,0,AU,1584137949,57,-23.85,151.25,75.20,10.29
8,Karasburg,0,NaN,1584137953,44,-28.02,18.75,70.79,11.99
9,Hualmay,30,PE,1584138006,54,-11.10,-77.61,80.69,10.20


In [18]:
# geocoordinates
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
radius = 5000
hotel_name = []


for index in range(len(vacay_df)):
    lat = hotels_df.iloc[index, 5]
    long = hotels_df.iloc[index, 6]
    location = f"{lat}, {long}"
    params = {"query": "hotel",
              "location": location,
              "radius": radius,
              "key": g_key}
    response = requests.get(base_url, params=params).json()
    hotel_name.append(response["results"][1]["name"])

In [20]:
hotels_df["Hotel Name"] = hotel_name
hotels_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Castro,0,BR,1584137354,47,-24.79,-50.01,75.83,3.60,Hotel Buganville Express
1,Mar del Plata,0,AR,1584137540,42,-38.00,-57.56,84.20,9.17,NH Gran Hotel Provincial
2,Praia,8,CV,1584137636,44,14.92,-23.51,78.80,14.99,Michael's Cape Verde Retreat
3,Point Fortin,20,TT,1584137823,51,10.18,-61.68,86.00,14.99,Clifton Hill Manor Hotel
4,Palāsa,0,IN,1584137857,52,18.77,84.42,73.49,1.83,Hotels at kssibugga
5,Mount Isa,0,AU,1584137920,46,-20.73,139.50,73.40,6.93,ibis Styles Mt Isa Verona
6,Valparaíso,10,CL,1584137717,49,-33.04,-71.63,73.40,13.87,ibis Valparaiso
7,Gladstone,0,AU,1584137949,57,-23.85,151.25,75.20,10.29,Rydges Gladstone
8,Karasburg,0,NaN,1584137953,44,-28.02,18.75,70.79,11.99,Avondsrus Guesthouse
9,Hualmay,30,PE,1584138006,54,-11.10,-77.61,80.69,10.20,Hotel Villa Kitzia Huacho


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map